(example-ohsome)=
# Ohsome API Test

Quellen:
[HeiGIT](https://heigit.org/big-spatial-data-analytics-en/ohsome/)

[ohsome quality analyst (OQT)](https://heigit.org/big-spatial-data-analytics-en/ohsome/ohsome-quality-analyst-oqt/)

API Endpoints:

In [ ]:
base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/length"
ohsome_url = base_url + endpoint


base_url = "https://api.quality.ohsome.org/v1"
endpoint = "/indicators"
indicator = "/mapping-saturation/data"
oqt_url = base_url + endpoint + indicator

In [ ]:
import json
import pprint as pp
from datetime import datetime
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import osmnx as ox
import pyproj # we need this for jupyter-book to build right...
import plotly.graph_objects as go
import plotly.io as pio
import requests
from IPython.display import SVG, display
from termcolor import colored

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def jsonKeys2int(x):
    """Convert json keys to int"""
    if isinstance(x, dict):
        x = {(int(k) if (k != "inf" and k != "self" and k !=
              "controll") else k): v for k, v in x.items()}
    return x

In [ ]:
def get_bicycle_filter():
    return "(highway=cycleway\
 or cycleway in (lane,crossing,track,shared_lane,opposite,shared,opposite_lane,yes,share_busway,sidepath,opposite_track)\
 or cycleway:left in (lane,crossing,track,shared_lane,opposite,shared,opposite_lane,yes,share_busway,sidepath,opposite_track)\
 or cycleway:right in (lane,crossing,track,shared_lane,opposite,shared,opposite_lane,yes,share_busway,sidepath,opposite_track)\
 or bicycle in (yes,designated,use_sidepath,permissive,official,optional_sidepath)\
 or living_street=yes)"

In [ ]:
def bpolys_from_name(name, which_result):
    df = ox.geocoder.geocode_to_gdf(name, which_result=which_result)
    north = df.bbox_north[0]
    east = df.bbox_east[0]
    south = df.bbox_south[0]
    west = df.bbox_west[0]
    # bpolys = {
    #     "type": "Feature",
    #     "properties": {"id": "Region 1"},
    #     "geometry": {
    #         "type": "Polygon",
    #         "coordinates": [
    #             [
    #                 [
    #                     east, north
    #                 ],
    #                 [
    #                     west, north
    #                 ],
    #                 [
    #                     west, south
    #                 ],
    #                 [
    #                     east, south
    #                 ],
    #                 [
    #                     east, north
    #                 ]
    #             ]
    #         ]
    #     }
    # }
    bpolys = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {"id": "Region 1"},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [east, north],
                            [west, north],
                            [west, south],
                            [east, south],
                            [east, north]
                        ]
                    ]
                }
            }
        ]
    }
    return bpolys

In [ ]:
def display_oqt_results(feature, name):
    label = feature["result"][0]["result"]["label"]
    color = feature["result"][0]["result"]["label"]
    description = feature["result"][0]["result"]["description"]
    # svg = feature["result"][0]["result"]["svg"]

    # print(json.dumps(result, indent=4))
    print(name)
    print("-" * len(name))
    print("Quality Label: " + colored(label, color))
    print("Result Description: " + description)
    # parse plotly figure
    result = feature["result"][0]["result"]
    figure = go.Figure(result["figure"])
    # display plotly figure
    display(figure)


def display_ohsome_results(result, filter_string):
    dates = []
    values = []
    for r in result:
        year = datetime.fromisoformat(r["timestamp"].replace("Z", ""))
        dates.append(year)
        values.append(r["value"])

    filter_string_short = filter_string[:20]
    fig, ax = plt.subplots()
    ax.plot(dates, values, label=filter_string_short)
    ax.legend()
    fig.suptitle("Temporal Evolution of {}".format(filter_string_short))
    plt.show()

In [ ]:
def get_ohsome_result(url="https://api.ohsome.org/v1/elements/length", name="Freiberg, Sachsen", filter_string=None):

    bbox = ox.geocoder.geocode_to_gdf(name, which_result=0)
    north = bbox.bbox_north[0]
    east = bbox.bbox_east[0]
    south = bbox.bbox_south[0]
    west = bbox.bbox_west[0]

    parameters = {
        "bboxes": "{},{},{},{}".format(west, south, east, north),
        "filter": "{}".format(filter_string),
        "format": "json",
        "time": "2008-01-01/2023-01-01/P1M",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    # print(json.dumps(parameters))
    response = requests.post(url, data=parameters, headers=headers)
    # print(response)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    ohsome_result = response.json()["result"]
    return ohsome_result

In [ ]:
def get_oqt_result(url="https://api.quality.ohsome.org/v1/indicators/mapping-saturation/data", bpolys=None, ohsome_result=None, filter_string=None):

    topic = {
        "key": "mapping-saturation-cycleways",
        "name": "Cycleways",
        "description": "Number of features taged as {}".format(filter_string),
        "data": {"result": ohsome_result}
    }

    parameters = {
        "bpolys": bpolys,
        "topic": topic,
        "include_figure": True
        # "includeHtml": False,
        # "flatten": False
    }

    # print(json.dumps(parameters))
    headers = {"Content-Type": "application/json",
               "Accept": "application/json"}
    response = requests.post(url, json=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    oqt_result = response.json()
    return oqt_result

In [ ]:
def get_results(name="Freiberg, Sachsen", which_result=0, filter_string=None):
    results = {}
    bpolys = bpolys_from_name(name=name, which_result=which_result)
    print(bpolys)
    ohsome_result = get_ohsome_result(name=name, filter_string=filter_string)
    print(ohsome_result)
    results["highest_count"] = ohsome_result[-1]["value"]
    # display_ohsome_results(ohsome_result)
    oqt_result = get_oqt_result(
        bpolys=bpolys, ohsome_result=ohsome_result, filter_string=filter_string)
    print(oqt_result)
    results["quality_label"] = oqt_result["result"][0]["result"]["label"]
    results["quality_description"] = oqt_result["result"][0]["result"]["description"]
    results["quality_value"] = oqt_result["result"][0]["result"]["value"]
    # display_oqt_results(oqt_result, name=name)
    return results

In [ ]:
def get_results_plot(name="Freiberg, Sachsen", which_result=0, filter_string=None):
    results = {}
    bpolys = bpolys_from_name(name=name, which_result=which_result)
    ohsome_result = get_ohsome_result(name=name, filter_string=filter_string)
    results["highest_count"] = ohsome_result[-1]["value"]
    display_ohsome_results(ohsome_result, filter_string=filter_string)
    oqt_result = get_oqt_result(
        bpolys=bpolys, ohsome_result=ohsome_result, filter_string=filter_string)
    results["quality_label"] = oqt_result["result"][0]["result"]["label"]
    results["quality_description"] = oqt_result["result"][0]["result"]["description"]
    results["quality_value"] = oqt_result["result"][0]["result"]["value"]
    display_oqt_results(oqt_result, name=name)
    return results

In [ ]:
name = "Freiberg, Sachsen"
which_result = 0
filter_string = "highway=* and geometry:line"
results = {}
bpolys = bpolys_from_name(name=name, which_result=which_result)
ohsome_result = get_ohsome_result(name=name, filter_string=filter_string)
results["highest_count"] = ohsome_result[-1]["value"]
# display_ohsome_results(ohsome_result, filter_string=filter_string)
oqt_result = get_oqt_result(
    bpolys=bpolys, ohsome_result=ohsome_result, filter_string=filter_string)
oqt_result

In [ ]:
oqt_result["result"][0]["result"]["figure"]

In [ ]:
# parse plotly figure
figure = go.Figure(oqt_result["result"][0]["result"]["figure"])
# display plotly figure
display(figure)

# convert figure to SVG
# svg = pio.to_image(figure, format='svg')
# display(SVG(svg))

{ city : {
    highway_type: {
        result_no_tag : result
        "tag": {
            result_tag : result
            percentage: result
        }
    }
}

In [ ]:
# filter_string = "(highway=cycleway or cycleway:both=* or cycleway:left=* or cycleway:right=*) and geometry:line"
# get_results(name="Freiberg, Sachsen", which_result=0)

def get_city_results(city="Freiberg, Sachsen", primary_filter="highway=*", secondary_filters=None):
    filter_base_line = "{0} and geometry:line".format(primary_filter)
    results_base_line = get_results(name=city, filter_string=filter_base_line)
    tags_dict = {}
    tag_dict = {}
    filter_all = "{0} and geometry:line".format(primary_filter)

    for secondary_filter in secondary_filters:

        filter_with_tag = "{0} and {1} and geometry:line".format(
            primary_filter, secondary_filter)
        result_with_tag = get_results(name=city, filter_string=filter_with_tag)
        print("base: {}".format(results_base_line))
        print("filtered: {}".format(result_with_tag))
        percentage_with_tag = (result_with_tag.get("highest_count", 0) /
                               results_base_line.get("highest_count", 1)) * 100

        tag_dict[secondary_filter] = {
            "results_with_tag": result_with_tag,
            "percentage_with_tag": percentage_with_tag
        }
        filter_all += " and {0}".format(secondary_filter)

    result_with_all_tag = get_results(name=city, filter_string=filter_all)
    percentage_with_all_tag = (result_with_all_tag.get("highest_count", 0) /
                               results_base_line.get("highest_count", 1)) * 100

    tag_dict[filter_all] = {
        "results_with_tag": result_with_all_tag,
        "percentage_with_tag": percentage_with_all_tag
    }

    tag_dict["results_no_tag"] = results_base_line
    tags_dict[primary_filter] = tag_dict
    return tags_dict
    # print("% Ways with {} tag {}".format(secondary_filter, coverage))
    # filter_string = "highway=* and width=* and geometry:line"
    # result_with_width = get_results(name="Freiberg, Sachsen", which_result=0,
    #                                 filter_string=filter_string)

    # IDEA: What percentage of Ways has all relevant tags?

In [ ]:
def get_city(city="Freiberg, Sachsen", primary_filter="highway=*", secondary_filters=["width=*", "smoothness=*", "surface=*", "lit=*"], force=False):
    city_file = Path("cache", "ohsome_results" + city + ".json")
    if city_file.exists() and not force:
        result_city = dict(
            json.load(open(city_file)))
    else:
        result_city = get_city_results(city, primary_filter, secondary_filters)
        json.dump(result_city, open(city_file, 'w'))
    return result_city

In [ ]:
def plot_donut(ax, percentage, secondary_text, highest_count=None, radius=0.25, width=0.15):
    # Determine the color based on the percentage
    if percentage < 25:
        colors = ['#ff0000', '#f0f0f0']  # red
    elif percentage <= 97:
        colors = ['#ffcc00', '#f0f0f0']  # yellow
    else:
        colors = ['#4caf50', '#f0f0f0']  # green

    # Values for the pie chart (donut)
    values = [percentage, 100 - percentage]
    # Plot the pie chart
    wedges, _ = ax.pie(values, colors=colors, startangle=90,
                       counterclock=False, radius=radius, wedgeprops={'width': width})
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax.axis('equal')
    # Add the secondary text
    ax.text(0, 0, f'{secondary_text}', ha='center', va='center', fontsize=10)
    # Add the highest_count text if provided
    if highest_count is not None:
        ax.text(0, -0.4, f'Count: {highest_count:,.2f}',
                ha='center', va='center', fontsize=8)

In [ ]:
def plot_city(city="Freiberg, Sachsen", ohsome_results=None):
    rows_data = ohsome_results[city]["highway=*"]
    # Create a figure and axes to plot the donut charts with any number of rows
    fig, axes = plt.subplots(len(rows_data) - 1, 3,
                             figsize=(12, len(rows_data) * 4))
    plt.subplots_adjust(wspace=0.5, hspace=0.7)
    # Adjusting the main title position
    plt.suptitle(city, fontsize=16, y=1.05)

    # Iterate through the data and plot the donut charts for each row
    for i, (key, values) in enumerate(rows_data.items()):

        if key == 'results_no_tag':
            continue
        # Extract values for plotting
        results_no_tag = ohsome_results[city]["highway=*"]['results_no_tag']
        results_with_tag = values['results_with_tag']
        percentage_with_tag = values['percentage_with_tag']

        # Plot the donut charts
        plot_donut(axes[i][0], results_no_tag['quality_value'] * 100,
                   f"{results_no_tag['quality_value'] * 100:.2f}%", highest_count=results_no_tag['highest_count'])
        plot_donut(axes[i][1], results_with_tag['quality_value'] * 100,
                   f"{results_with_tag['quality_value'] * 100:.2f}%", highest_count=results_with_tag['highest_count'])
        plot_donut(axes[i][2], percentage_with_tag,
                   f"{percentage_with_tag:.2f}%")

        # Adding the titles for the rows
        # Adjusting the position
        axes[i][1].set_title(key, fontsize=14, y=1.05)

    plt.show()

## Ergebnisse

Zu sehen sind:
- 1. Spalte die Sättigung des primary_filters. In diesem Fall "highway=* and geometry:line". Das umfasst alle Straßen die in OSM eingetragen sind.
- 2. Spalte ist die Sättigung inklusive des secondary_filters jeweils durch die Überschrift der Zeile angezeigt. z.B.(width=* würde als Filter haben "highway=* and geometry:line and width=*")
- 3. Spalte ist das Prozentualle Verhältniss zwischen der maximalen Anzahl von Objekten ohne secondary_filter zu der Anzahl der Objekte mit secondary_filter. Das heißt wie viel Prozent der Straßen dieses Attribut besitzen.

## Interpretation

Ist eine Sättigung noch nicht erreicht, so sind Freiwillige noch dabei dieses Tag zu vervollständigen. Es ist dann nicht plausibel eine Aussage über die Daten zu treffen, da klar ist, dass sie noch nicht vollständig sind.
Ist die Sättigung erreicht, so ist es unwahrscheinlich, dass sich noch jemand die Mühe macht dieses Tag zu vervollständigen.

```{note}
Nur weil die Sättigung hoch ist bedeutet das nicht, dass alle Attribute/Straßen vollständig eingetragen sind. Es heißt nur, das keine neuen mehr eingetragen werden.
Das kann verschiedene Gründe haben. Zum Beispiel, kann es sein, dass Leute sich den Aufwand nicht machen wollen, oder es gibt keine Datenquellen für das Attribut.
```

Ist die erste Spalte gesättigt und die zweite nicht, wird sich das Verhältniss in der dritten Spalte noch verbessern.

Sind Spalte eins und Spalte zwei gesättigt, so wird sich das Verhältniss in der dritten Spalte nicht mehr verbessern. Das bedeutet, es wird bei der angegebenen Abdeckung bleiben.
Als Beispiel Freiberg alle Straßen mit Breite: 97.26%, 93,44% und 5,08%.
Sättigung der Straßen ist erreicht, es kann also eine Aussage getroffen werden.
Sättigung des "width" ist zwar noch nicht Erreicht aber schon relativ hoch. Es ist also davon auszugehen, dass sich das Verhältniss nicht mehr groß ändern wird.
Das Verhältniss ist 5,08% der Straßen haben eine eingetragende Breite. Das ist sehr wenig und eignet sich nicht für eine Bewertung R4R.

In [ ]:
#     "timestamp": "201{"bpolys": {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {"id": "Region 1"}, "geometry": {"type": "Polygon", "coordinates": [[[13.4005105, 50.9590788], [13.2660828, 50.9590788], [13.2660828, 50.8754646], [13.4005105, 50.8754646], [13.4005105, 50.9590788]]]}}]}, "topic": {"key": "mapping-saturation", "name": "Cycleways", "description": "Number of features taged as highway=* and geometry:line", "data": {"result": [{"timestamp": "2008-01-01T00:00:00Z", "value": 87197.26}, {"timestamp": "2008-02-01T00:00:00Z", "value": 93851.79}, {"timestamp": "2008-03-01T00:00:00Z", "value": 93345.01}, {"timestamp": "2008-04-01T00:00:00Z", "value": 112622.76}, {"timestamp": "2008-05-01T00:00:00Z", "value": 154680.48}, {"timestamp": "2008-06-01T00:00:00Z", "value": 156881.88}, {"timestamp": "2008-07-01T00:00:00Z", "value": 214636.06}, {"timestamp": "2008-08-01T00:00:00Z", "value": 300421.42}, {"timestamp": "2008-09-01T00:00:00Z", "value": 326685.24}, {"timestamp": "2008-10-01T00:00:00Z", "value": 332218.54}, {"timestamp": "2008-11-01T00:00:00Z", "value": 336596.65}, {"timestamp": "2008-12-01T00:00:00Z", "value": 348372.79}, {"timestamp": "2009-01-01T00:00:00Z", "value": 352830.71}, {"timestamp": "2009-02-01T00:00:00Z", "value": 356348.66}, {"timestamp": "2009-03-01T00:00:00Z", "value": 357086.9}, {"timestamp": "2009-04-01T00:00:00Z", "value": 367789.44}, {"timestamp": "2009-05-01T00:00:00Z", "value": 386157.49}, {"timestamp": "2009-06-01T00:00:00Z", "value": 427495.08}, {"timestamp": "2009-07-01T00:00:00Z", "value": 434106.91}, {"timestamp": "2009-08-01T00:00:00Z", "value": 439900.83}, {"timestamp": "2009-09-01T00:00:00Z", "value": 439590.67}, {"timestamp": "2009-10-01T00:00:00Z", "value": 455880.06}, {"timestamp": "2009-11-01T00:00:00Z", "value": 457644.74}, {"timestamp": "2009-12-01T00:00:00Z", "value": 462019.69}, {"timestamp": "2010-01-01T00:00:00Z", "value": 468501.67}, {"timestamp": "2010-02-01T00:00:00Z", "value": 471007.62}, {"timestamp": "2010-03-01T00:00:00Z", "value": 473570.51}, {"timestamp": "2010-04-01T00:00:00Z", "value": 477883.31}, {"timestamp": "2010-05-01T00:00:00Z", "value": 506958.02}, {"timestamp": "2010-06-01T00:00:00Z", "value": 514824.03}, {"timestamp": "2010-07-01T00:00:00Z", "value": 520368.68}, {"timestamp": "2010-08-01T00:00:00Z", "value": 528538.92}, {"timestamp": "2010-09-01T00:00:00Z", "value": 533096.06}, {"timestamp": "2010-10-01T00:00:00Z", "value": 564183.29}, {"timestamp": "2010-11-01T00:00:00Z", "value": 568241.26}, {"timestamp": "2010-12-01T00:00:00Z", "value": 573445.51}, {"timestamp": "2011-01-01T00:00:00Z", "value": 603163.89}, {"timestamp": "2011-02-01T00:00:00Z", "value": 607813.83}, {"timestamp": "2011-03-01T00:00:00Z", "value": 610816.05}, {"timestamp": "2011-04-01T00:00:00Z", "value": 612901.46}, {"timestamp": "2011-05-01T00:00:00Z", "value": 620914.97}, {"timestamp": "2011-06-01T00:00:00Z", "value": 632676.3}, {"timestamp": "2011-07-01T00:00:00Z", "value": 636170.31}, {"timestamp": "2011-08-01T00:00:00Z", "value": 637438.7}, {"timestamp": "2011-09-01T00:00:00Z", "value": 639767.54}, {"timestamp": "2011-10-01T00:00:00Z", "value": 640739.99}, {"timestamp": "2011-11-01T00:00:00Z", "value": 641294.31}, {"timestamp": "2011-12-01T00:00:00Z", "value": 642568.07}, {"timestamp": "2012-01-01T00:00:00Z", "value": 644387.57}, {"timestamp": "2012-02-01T00:00:00Z", "value": 647950.67}, {"timestamp": "2012-03-01T00:00:00Z", "value": 651230.03}, {"timestamp": "2012-04-01T00:00:00Z", "value": 651203.76}, {"timestamp": "2012-05-01T00:00:00Z", "value": 652406.36}, {"timestamp": "2012-06-01T00:00:00Z", "value": 654956.67}, {"timestamp": "2012-07-01T00:00:00Z", "value": 657949.33}, {"timestamp": "2012-08-01T00:00:00Z", "value": 659719.81}, {"timestamp": "2012-09-01T00:00:00Z", "value": 659824.03}, {"timestamp": "2012-10-01T00:00:00Z", "value": 661694.82}, {"timestamp": "2012-11-01T00:00:00Z", "value": 662141.8}, {"timestamp": "2012-12-01T00:00:00Z", "value": 666633.18}, {"timestamp": "2013-01-01T00:00:00Z", "value": 669473.09}, {"timestamp": "2013-02-01T00:00:00Z", "value": 670184.03}, {"timestamp": "2013-03-01T00:00:00Z", "value": 671608.34}, {"timestamp": "2013-04-01T00:00:00Z", "value": 672589.53}, {"timestamp": "2013-05-01T00:00:00Z", "value": 672779.61}, {"timestamp": "2013-06-01T00:00:00Z", "value": 673862.15}, {"timestamp": "2013-07-01T00:00:00Z", "value": 677107.82}, {"timestamp": "2013-08-01T00:00:00Z", "value": 680645.68}, {"timestamp": "2013-09-01T00:00:00Z", "value": 685069.54}, {"timestamp": "2013-10-01T00:00:00Z", "value": 691901.8}, {"timestamp": "2013-11-01T00:00:00Z", "value": 698495.29}, {"timestamp": "2013-12-01T00:00:00Z", "value": 701754.2}, {"timestamp": "2014-01-01T00:00:00Z", "value": 704175.97}, {"timestamp": "2014-02-01T00:00:00Z", "value": 708558.55}, {"timestamp": "2014-03-01T00:00:00Z", "value": 710839.34}, {"timestamp": "2014-04-01T00:00:00Z", "value": 710616.09}, {"timestamp": "2014-05-01T00:00:00Z", "value": 713205.13}, {"timestamp": "2014-06-01T00:00:00Z", "value": 717905.62}, {"timestamp": "2014-07-01T00:00:00Z", "value": 723354.98}, {"timestamp": "2014-08-01T00:00:00Z", "value": 724143.4}, {"timestamp": "2014-09-01T00:00:00Z", "value": 744447.23}, {"timestamp": "2014-10-01T00:00:00Z", "value": 747572.1}, {"timestamp": "2014-11-01T00:00:00Z", "value": 748113.9}, {"timestamp": "2014-12-01T00:00:00Z", "value": 748213.46}, {"timestamp": "2015-01-01T00:00:00Z", "value": 749825.81}, {"timestamp": "2015-02-01T00:00:00Z", "value": 749964.44}, {"timestamp": "2015-03-01T00:00:00Z", "value": 750733.46}, {
# 5-04-01T00:00:00Z", "value": 751018.61}, {"timestamp": "2015-05-01T00:00:00Z", "value": 752132.27}, {"timestamp": "2015-06-01T00:00:00Z", "value": 752141.86}, {"timestamp": "2015-07-01T00:00:00Z", "value": 753028.42}, {"timestamp": "2015-08-01T00:00:00Z", "value": 753882.29}, {"timestamp": "2015-09-01T00:00:00Z", "value": 754210.39}, {"timestamp": "2015-10-01T00:00:00Z", "value": 755551.22}, {"timestamp": "2015-11-01T00:00:00Z", "value": 759191.66}, {"timestamp": "2015-12-01T00:00:00Z", "value": 734134.92}, {"timestamp": "2016-01-01T00:00:00Z", "value": 736295.3}, {"timestamp": "2016-02-01T00:00:00Z", "value": 741799.3}, {"timestamp": "2016-03-01T00:00:00Z", "value": 765884.89}, {"timestamp": "2016-04-01T00:00:00Z", "value": 767946.28}, {"timestamp": "2016-05-01T00:00:00Z", "value": 771447.65}, {"timestamp": "2016-06-01T00:00:00Z", "value": 771610.29}, {"timestamp": "2016-07-01T00:00:00Z", "value": 772049.5}, {"timestamp": "2016-08-01T00:00:00Z", "value": 772020.12}, {"timestamp": "2016-09-01T00:00:00Z", "value": 772020.12}, {"timestamp": "2016-10-01T00:00:00Z", "value": 774482.43}, {"timestamp": "2016-11-01T00:00:00Z", "value": 775371.05}, {"timestamp": "2016-12-01T00:00:00Z", "value": 775543.77}, {"timestamp": "2017-01-01T00:00:00Z", "value": 775553.49}, {"timestamp": "2017-02-01T00:00:00Z", "value": 775860.09}, {"timestamp": "2017-03-01T00:00:00Z", "value": 775859.48}, {"timestamp": "2017-04-01T00:00:00Z", "value": 775848.16}, {"timestamp": "2017-05-01T00:00:00Z", "value": 776000.91}, {"timestamp": "2017-06-01T00:00:00Z", "value": 776566.72}, {"timestamp": "2017-07-01T00:00:00Z", "value": 774786.12}, {"timestamp": "2017-08-01T00:00:00Z", "value": 774806.04}, {"timestamp": "2017-09-01T00:00:00Z", "value": 775693.95}, {"timestamp": "2017-10-01T00:00:00Z", "value": 776242.36}, {"timestamp": "2017-11-01T00:00:00Z", "value": 778052.8}, {"timestamp": "2017-12-01T00:00:00Z", "value": 778821.73}, {"timestamp": "2018-01-01T00:00:00Z", "value": 779100.51}, {"timestamp": "2018-02-01T00:00:00Z", "value": 779495.48}, {"timestamp": "2018-03-01T00:00:00Z", "value": 779740.79}, {"timestamp": "2018-04-01T00:00:00Z", "value": 779933.85}, {"timestamp": "2018-05-01T00:00:00Z", "value": 787797.88}, {"timestamp": "2018-06-01T00:00:00Z", "value": 787930.9}, {"timestamp": "2018-07-01T00:00:00Z", "value": 788526.29}, {"timestamp": "2018-08-01T00:00:00Z", "value": 788322.3}, {"timestamp": "2018-09-01T00:00:00Z", "value": 788340.77}, {"timestamp": "2018-10-01T00:00:00Z", "value": 788601.71}, {"timestamp": "2018-11-01T00:00:00Z", "value": 790041.51}, {"timestamp": "2018-12-01T00:00:00Z", "value": 799565.33}, {"timestamp": "2019-01-01T00:00:00Z", "value": 805412.08}, {"timestamp": "2019-02-01T00:00:00Z", "value": 806154.95}, {"timestamp": "2019-03-01T00:00:00Z", "value": 811278.66}, {"timestamp": "2019-04-01T00:00:00Z", "value": 811673.81}, {"timestamp": "2019-05-01T00:00:00Z", "value": 812378.91}, {"timestamp": "2019-06-01T00:00:00Z", "value": 812988.82}, {"timestamp": "2019-07-01T00:00:00Z", "value": 815458.23}, {"timestamp": "2019-08-01T00:00:00Z", "value": 815903.77}, {"timestamp": "2019-09-01T00:00:00Z", "value": 816434.96}, {"timestamp": "2019-10-01T00:00:00Z", "value": 817038.12}, {"timestamp": "2019-11-01T00:00:00Z", "value": 817427.87}, {"timestamp": "2019-12-01T00:00:00Z", "value": 817983.83}, {"timestamp": "2020-01-01T00:00:00Z", "value": 821818.56}, {"timestamp": "2020-02-01T00:00:00Z", "value": 824176.55}, {"timestamp": "2020-03-01T00:00:00Z", "value": 824769.74}, {"timestamp": "2020-04-01T00:00:00Z", "value": 826663.23}, {"timestamp": "2020-05-01T00:00:00Z", "value": 827651.24}, {"timestamp": "2020-06-01T00:00:00Z", "value": 829257.35}, {"timestamp": "2020-07-01T00:00:00Z", "value": 830951.0}, {"timestamp": "2020-08-01T00:00:00Z", "value": 831549.22}, {"timestamp": "2020-09-01T00:00:00Z", "value": 833823.46}, {"timestamp": "2020-10-01T00:00:00Z", "value": 834292.91}, {"timestamp": "2020-11-01T00:00:00Z", "value": 834472.08}, {"timestamp": "2020-12-01T00:00:00Z", "value": 834941.17}, {"timestamp": "2021-01-01T00:00:00Z", "value": 836737.02}, {"timestamp": "2021-02-01T00:00:00Z", "value": 837550.02}, {"timestamp": "2021-03-01T00:00:00Z", "value": 843717.15}, {"timestamp": "2021-04-01T00:00:00Z", "value": 843875.74}, {"timestamp": "2021-05-01T00:00:00Z", "value": 843838.3}, {"timestamp": "2021-06-01T00:00:00Z", "value": 844450.57}, {"timestamp": "2021-07-01T00:00:00Z", "value": 844399.06}, {"timestamp": "2021-08-01T00:00:00Z", "value": 844636.7}, {"timestamp": "2021-09-01T00:00:00Z", "value": 844627.64}, {"timestamp": "2021-10-01T00:00:00Z", "value": 844718.25}, {"timestamp": "2021-11-01T00:00:00Z", "value": 844810.92}, {"timestamp": "2021-12-01T00:00:00Z", "value": 845905.92}, {"timestamp": "2022-01-01T00:00:00Z", "value": 847528.74}, {"timestamp": "2022-02-01T00:00:00Z", "value": 848498.84}, {"timestamp": "2022-03-01T00:00:00Z", "value": 850203.8}, {"timestamp": "2022-04-01T00:00:00Z", "value": 848101.36}, {"timestamp": "2022-05-01T00:00:00Z", "value": 847712.18}, {"timestamp": "2022-06-01T00:00:00Z", "value": 848878.23}, {"timestamp": "2022-07-01T00:00:00Z", "value": 849125.62}, {"timestamp": "2022-08-01T00:00:00Z", "value": 856711.08}, {"timestamp": "2022-09-01T00:00:00Z", "value": 856705.32}, {"timestamp": "2022-10-01T00:00:00Z", "value": 857658.21}, {"timestamp": "2022-11-01T00:00:00Z", "value": 858757.51}, {"timestamp": "2022-12-01T00:00:00Z", "value": 859361.89}, {"timestamp": "2023-01-01T00:00:00Z", "value": 860655.16}]}}, "includeSvg": true, "includeHtml": false, "flatten": false}

In [ ]:
ohsome_results = {}
city = "Freiberg, Sachsen"
ohsome_results[city] = get_city(city)
# print(get_city().to_json())

In [ ]:
plot_city(city, ohsome_results)

In [ ]:
ohsome_results

In [ ]:
city = "Dresden"
ohsome_results[city] = get_city(city)

In [ ]:
plot_city(city, ohsome_results)

In [ ]:
city = "Leipzig"
ohsome_results[city] = get_city(city)

In [ ]:
plot_city(city, ohsome_results)

## Sättigungskurven am Beispiel von Freiberg
### Alle Wege/Straßen

In [ ]:
primary_filter = "highway=*"
secondary_filter = "width=*"
name = "Freiberg, Sachsen"
filter_with_tag = "{0} and {1} and geometry:line".format(
    primary_filter, secondary_filter)
filter_no_tag = "{0} and geometry:line".format(primary_filter)

In [ ]:
get_results_plot(name, filter_string=filter_no_tag)

In [ ]:
# get_results_plot(name, filter_string=filter_with_tag)

### Nur Radwege

In [ ]:
primary_filter = "highway=cycleway or cycleway=*"
secondary_filter = "width=*"
name = "Freiberg, Sachsen"
filter_with_tag = "{0} and {1} and geometry:line".format(
    primary_filter, secondary_filter)
filter_no_tag = "{0} and geometry:line".format(primary_filter)
get_results_plot(name, filter_string=filter_no_tag)

In [ ]:
primary_filter = get_bicycle_filter()
secondary_filter = "width=*"
name = "Freiberg, Sachsen"
filter_with_tag = "{0} and {1} and geometry:line".format(
    primary_filter, secondary_filter)
filter_no_tag = "{0} and geometry:line".format(primary_filter)

In [ ]:
filter_with_tag

In [ ]:
get_results_plot(name, filter_string=filter_no_tag)

In [ ]:
get_results_plot(name, filter_string=filter_with_tag)

## Sättigungskurven am Beispiel von Dresden
### Alle Wege/Straßen

In [ ]:
primary_filter = "highway=*"
secondary_filter = "width=*"
name = "Dresden"
filter_with_tag = "{0} and {1} and geometry:line".format(
    primary_filter, secondary_filter)
filter_no_tag = "{0} and geometry:line".format(primary_filter)

In [ ]:
get_results_plot(name, filter_string=filter_no_tag)

In [ ]:
get_results_plot(name, filter_string=filter_with_tag)

### Nur Radwege

In [ ]:
primary_filter = get_bicycle_filter()
secondary_filter = "width=*"
name = "Dresden"
filter_with_tag = "{0} and {1} and geometry:line".format(
    primary_filter, secondary_filter)
filter_no_tag = "{0} and geometry:line".format(primary_filter)

In [ ]:
get_results_plot(name, filter_string=filter_no_tag)

In [ ]:
get_results_plot(name, filter_string=filter_with_tag)